# ETL Lab

### Introduction

In this lab, we ask you to use the techniques learned in this section to work with an API of your choosing.  As important to getting to the correct code is to develop the proper procedure for getting there.  Just like in the preceding lessons we will following our procedures such as: 

1. Red, green, refactor
2. Move mess into an object
3. Make small methods by: 
    A. Commenting code
    B. Translating comments into methods
    
Along the way, we will arrive at our pattern of a *Client*, *Adapter*, and *Target*.

### Step 1.  Just get the data

The first step is to go from red to green.  That is, the code starts off with nothing working and our task is simply to get it working.  In this case, this means the following: 

1. Call an API of your choosing
2. Return a list of dictionaries and store as a variable named `entities`

In [1]:
import requests
url = 'https://data.austintexas.gov/resource/nguv-n54k.json'
response = requests.get(url)
entities = response.json()

In [2]:
type(entities)
# list

type(entities[0])

dict

### Step 2. Change the dictionaries into objects

The next step is to change dictionaries received back from the API into objects.  We can break this down into a couple of steps.

1. Create the *target class*.  This is the class the dictionaries will be transformed into.  To do this, choose no more than five attributes to store in each instance.

In [3]:
class Restaurant:
    # please change the name of this class
    def __init__(self,restaurant_name,zip,inspection_date,score):
        self._restaurant_name = restaurant_name
        self._zip = zip
        self._inspection_date = inspection_date
        self._score = score

Check your work by assigning an instance to the variable `target_instance`.

In [4]:
target_instance = Restaurant(entities[0]['restaurant_name'],entities[0]['zip_code'],entities[0]['inspection_date'],entities[0]['score'])
3 < len( target_instance.__dict__.keys()) < 5
# True 

True

1. Reject some of the data

We don't want to pass all of our data into our class.  So create a smaller dictionary of just the attributes we need.

In [5]:
selected_attributes = {'restaurant_name':entities[0]['restaurant_name'],'zip':entities[0]['zip_code'],
                       'inspection_date':entities[0]['inspection_date'],'score':entities[0]['score']}

In [6]:
type(selected_attributes)
# dict

dict

In [7]:
len(selected_attributes.keys()) == len(target_instance.__dict__.keys())
# True

True

2. Coerce dictionaries into objects

A. To start, coerce just one dictionary into an object.

In [8]:
first_object = Restaurant(selected_attributes['restaurant_name'],selected_attributes['zip'],selected_attributes['inspection_date'],selected_attributes['score'])
# change the above line to reference your target class

In [9]:
list(first_object.__dict__.values()) == list(entities[0].values())
# True

False

B. Now that you have solved for one, solve for all.  Coerce all of the dictionaries into objects.  Assign the list of objects to a variable `targets`.

In [12]:
targets = []
for result in range(len(entities)):
    target = {'restaurant_name':entities[result]['restaurant_name'], 'zip':entities[result]['zip_code'],
              'inspection_date':entities[result]['inspection_date'], 'score':entities[result]['score']}
    targets.append(target)
targets

[{'restaurant_name': 'Doubletree Club Hotel',
  'zip': '78702',
  'inspection_date': '2019-04-11T19:00:00.000',
  'score': '94'},
 {'restaurant_name': 'Omelettry, The',
  'zip': '78751',
  'inspection_date': '2019-04-10T19:00:00.000',
  'score': '87'},
 {'restaurant_name': 'Mobile Loaves and Fishes - Community First Village',
  'zip': '78724',
  'inspection_date': '2019-04-11T19:00:00.000',
  'score': '90'},
 {'restaurant_name': 'Wal-Mart Supercenter',
  'zip': '78748',
  'inspection_date': '2019-04-08T19:00:00.000',
  'score': '97'},
 {'restaurant_name': 'Howdy Donut',
  'zip': '78746',
  'inspection_date': '2019-04-10T19:00:00.000',
  'score': '93'},
 {'restaurant_name': 'Donut 7',
  'zip': '78758',
  'inspection_date': '2019-04-10T19:00:00.000',
  'score': '78'},
 {'restaurant_name': 'International House of Pancakes',
  'zip': '78751',
  'inspection_date': '2019-04-04T19:00:00.000',
  'score': '92'},
 {'restaurant_name': 'Del Valle Elementary School',
  'zip': '78617',
  'inspection

In [13]:
len(targets) == len(entities)
# True

True

### 3. Move the remaining code into an object 

At this point, we successfully have transformed a list of dictionaries from an API into a list of objects.  But we need to keep cleaning up our code.  To do this, look at the code outside of a class, and move it into a class, with a method name of run.

In [14]:
class RestaurantAdapter:
    def __init__(self, restaurant_dicts):
        self._restaurant_dicts = restaurant_dicts
        
    def run(self):
        self._targets = []
        for restaurant_dict in self._restaurant_dicts:
            selected_attribute = {'restaurant_name':restaurant_dict['restaurant_name'], 
                                  'zip':restaurant_dict['zip_code'],
                                  'inspection_date':restaurant_dict['inspection_date'], 
                                  'score':restaurant_dict['score']}
            target = Restaurant(selected_attribute['restaurant_name'],selected_attribute['zip'],
                                selected_attribute['inspection_date'],selected_attribute['score'])
            self._targets.append(target)
        return self._targets

Let's marke sure that this works.

In [15]:
adapter = RestaurantAdapter(entities)
# change the above line to reference your adapter
results = adapter.run()

In [16]:
len(results) == len(targets)
# True

True

### 4. Make the methods smaller

Next, separate out the run method in the adapter class smaller.  Do this by first writing comments in the code, and then moving the code into separate methods.  Please leave the comments in your code.  Your methods should be no longer than five lines long, and there can only be a total of one `if else` statement or `loop` per method.  Having both an `if else` and a `loop` in any method is also too complicated -- don't do it.

In [17]:
class RestaurantAdapter:
    def __init__(self, restaurant_dicts):
        self._restaurant_dicts = restaurant_dicts
        
    def run(self):
        self._targets = []
        for restaurant_dict in self._restaurant_dicts:
            selected_attribute = {'restaurant_name':restaurant_dict['restaurant_name'], 
                                  'zip':restaurant_dict['zip_code'],
                                  'inspection_date':restaurant_dict['inspection_date'], 
                                  'score':restaurant_dict['score']}
            target = Restaurant(selected_attribute['restaurant_name'],selected_attribute['zip'],
                                selected_attribute['inspection_date'],selected_attribute['score'])
            self._targets.append(target)
        return self._targets

### 5. Create the client class

Next move calls to the API into their own separate class.  This way we can call the API but later to decide to coerce the data into different types of objects than we did above.

In [18]:
import requests
class Client:
    def run(self):
        url = 'https://data.austintexas.gov/resource/nguv-n54k.json'
        entities = requests.get(url).json()
        return entities

Place the updated Adapter class below.  Check that it still works as it did before.

In [19]:
class RestaurantAdapter:
    def __init__(self, restaurant_dicts):
        self._restaurant_dicts = restaurant_dicts
        
    def run(self):
        self._targets = []
        for restaurant_dict in self._restaurant_dicts:
            selected_attribute = {'restaurant_name':restaurant_dict['restaurant_name'], 
                                  'zip':restaurant_dict['zip_code'],
                                  'inspection_date':restaurant_dict['inspection_date'], 
                                  'score':restaurant_dict['score']}
            target = Restaurant(selected_attribute['restaurant_name'],selected_attribute['zip'],
                                selected_attribute['inspection_date'],selected_attribute['score'])
            self._targets.append(target)
        return self._targets

In [20]:
refactored_adapter = RestaurantAdapter(entities)

len(refactored_adapter.run()) == len(adapter.run())

True

### Summary

Great job!  Hopefully, you saw how by building our code and then slowly refactoring our code, we can eventually get to some clean code.